In [ ]:
import polars as pl

In [ ]:
df_t = pl.read_csv("tilsyn.csv", separator=";", dtypes={"postnr":str})
df_t

In [ ]:
# Vi kan skrive til CSV
df_t.write_csv("tilsyn2.csv")

In [ ]:
# Vi kan skrive til Parquet
df_t.write_parquet("tilsyn.parquet")

In [ ]:
#Vi kan velge en serie:
df_t["navn"]

In [ ]:
# Vi kan velge én eller flere kolonner - dette blir en ny DataFrame
df_t[["navn", "postnr"]]

In [ ]:
# Men helst så burde vi bruke denne syntaksen til å velge flere kolonner:
df_t.select("navn", "postnr")

In [ ]:
# Vi kan også velge noe som er et uttrykk - legg merke til at "A" repeteres
# pl.lit("A") betyr en literal
df_t.select(pl.lit("A"), "postnr")

In [ ]:
# Strengt tatt så kan vi bruke pl.col("postnr") for å si at vi vil ha kolonnen postnummer
df_t.select(pl.lit("A"), pl.col("postnr"))

In [ ]:
# Vi kan filtrere på uttrykk:
df_t.filter(pl.col("postnr") == pl.lit("3179"))

In [ ]:
# Dette endrer ikke df_t
df_t

In [ ]:
# Vi må gjøre assignment for å endre noe.
# Bruk ~ for negasjon, | for or og & for and. 
# Husk å sette parantes rundt uttrykkene.. 

df_t2 = df_t.filter((pl.col("postnr") == "3179") | (pl.col("poststed") == "HORTEN"))
df_t2

In [ ]:
# Vi kan tilordne til kolonner
df_t3 = df_t.with_columns(
    (pl.col("postnr") + pl.lit(" ") + pl.col("poststed")).alias("postnr poststed")
)
df_t3

In [ ]:
# Vi fikser opp i datoen
df_t = df_t.with_columns(
    pl.col("dato").cast(str).str.pad_start(8, "0").str.to_date(format="%d%m%Y")
)

In [ ]:
# Vi kan lage mer kompliserte uttrykk
# .str inneholder en del streng-metoder, viktig at datatypen er string i dette tilfellet.
df_t.with_columns(
    (pl.col("poststed").str.slice(0,1) + 
     pl.col("poststed").str.slice(1, None).str.to_lowercase()).alias("poststed"))

In [ ]:
df_k = pl.read_csv("kravpunkter.csv", separator=";").with_columns(
    pl.col("dato").cast(str).str.pad_start(8, "0").str.to_date(format="%d%m%Y")
)
df_k

In [ ]:
# Vi kan joine:
df_j = df_t.select(
    pl.col("navn"), pl.col("orgnummer"), pl.col("tilsynid")
    ).join(
    df_k.select(
        pl.col("tilsynid"), pl.col("ordningsverdi"), pl.col("karakter"), pl.col("kravpunktnavn_no")
    ), on="tilsynid",how="left")
df_j

In [ ]:
# Vi kan gruppere og aggregere
df_j.group_by("tilsynid", "orgnummer", "navn").agg(
    (pl.col("karakter") == 3).any().alias("noen_karakter_3")
)

In [ ]:
# Vi kan kombinere flere uttrykk
# Vi kan gruppere og aggregere og deretter sortere
df_j.group_by("tilsynid", "orgnummer", "navn").agg(
    (pl.col("karakter") == 3).any().alias("noen_karakter_3")
).sort("noen_karakter_3", descending=True)

In [ ]:
# Vi bruker .lazy() for å lage en spørringsplan, og .collect() for å eksekvere spørringsplanen. 
# Vi bruker .scan_csv() for å bare lage en plan om å lese CSV.
# Optimaliseringene gjør at vi kun leser inn de kolonnene som faktisk blir brukt!
# Dette kalles en LazyFrame, ikke en DataFrame
lf_t = pl.scan_csv("tilsyn.csv", separator=";", dtypes={"postnr":str})
df_t = lf_t.select("orgnummer", "navn", "dato", "total_karakter", "poststed").with_columns(
    pl.col("dato").cast(str).str.pad_start(8, "0").str.to_date(format="%d%m%Y")
).filter(
    pl.col("total_karakter").is_in([2,3])
).filter(
    pl.col("dato").dt.year() == 2018
).filter(
    pl.col("poststed") == "HORTEN"
).collect()
df_t